## Third Movement: 

In [14]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [15]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [16]:
%run attack_graph_MIR100.ipynb

### Use ??? Distribution

In [17]:
# The key change - enhanced random_steps function
attack_rate_list = [0]   # Not used but kept for API consistency
defense_rate_list = [0]  # Not used but kept for API consistency

In [18]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    """
    Calculate probability distribution based on edge difficulties.
    
    Args:
        route: List of nodes representing an attack path
        attack_rate: Not used in this implementation but kept for API consistency
        defense_rate: Not used in this implementation but kept for API consistency
        graph: NetworkX graph containing edge probabilities
        
    Returns:
        numpy array: Probability distribution for number of steps attacker can take
    """
    # Get edge probabilities between consecutive nodes in route
    hardness = []
    for i in range(len(route)-1):
        # Get all edges between these nodes (handles parallel edges)
        edges_data = graph.get_edge_data(route[i], route[i+1])
        
        # If multiple edges exist between nodes, take the maximum probability
        probs = []
        for key in edges_data:
            prob = edges_data[key].get('edge_probabilities', 1.0)
            probs.append(float(prob))
        
        # Take maximum probability if multiple edges exist
        hardness.append(max(probs))
    
    # Convert to numpy array and handle missing values
    hardness = np.array(hardness)
    hardness[np.isnan(hardness)] = 1.0  # Missing probabilities treated as certainty
    
    # Calculate PDF according to the paper's formula
    # P(n steps) = (1-p_{n+1}) * product(p_1 to p_n)
    # where p_i is probability of success at step i
    
    # Calculate cumulative products
    cumprod = np.concatenate(([1.0], np.cumprod(hardness)))
    
    # Calculate (1-p) terms
    one_minus_h = np.concatenate((1 - hardness, [1.0]))
    
    # Final PDF calculation
    pdf = one_minus_h * cumprod
    
    # Normalize to ensure it sums to 1
    pdf = pdf / np.sum(pdf)
    
    return pdf

In [19]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()

In [20]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Sat Dec 21 15:50:34 2024"

++++++++++++++++++++++++++++++++

The virtual target nodeID is 17

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
1 0.000000e+00
10 0.000000e+00
11 2.388662e-01
15 1.976793e-01
2 0.000000e+00
3 0.000000e+00
4 0.000000e+00
5 0.000000e+00
6 0.000000e+00
7 0.000000e+00
8 5.634545e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.0000000
3 0.0000000
4 0.3411805
5 0.0000000
6 0.0000000
7 0.1376046
8 0.0000000
9 0.0000000
10 0.0000000
11 0.0000000
12 0.0000000
13 0.0000000
14 0.0000000
15 0.0000000
16 0.0000000
17 0.0000000
18 0.0000000
19 0.0000000
20 0.0000000
21 0.0000000
22 0.0000000
23 0.5212148
24 0.0000000
25 0.0000000
26 0.0000000
[1] 0.085

Defender can keep attacker success below: 0.085
Attacker can guarantee success probability of: 0.085

